In [1]:
import pandas as pd
import numpy as np
import random
import copy
import math

from sklearn.linear_model import LogisticRegression

In [2]:
import pandas as pd
excel = pd.ExcelFile("data_cleaned.xlsx")
data = excel.parse(0)

In [3]:
# remove unnamed column
for i in data:
    if i.startswith('Unnamed') or i.startswith('CLINICAL OUTCOMES') or i.startswith('AUTO-CT'):
        data.pop(i)
    
data1 = data.dropna(axis = 0,how='all') # Remove Rows with All NAS

In [4]:
# print column name with corresponding number
for count, name in enumerate(data1):
    print(f'{count}:"{name}",')

0:"Clinical F/U interval  [d from CT]",
1:"BMI",
2:"BMI >30",
3:"Sex",
4:"Age at CT",
5:"Tobacco",
6:"FRS 10-year risk (%)",
7:"FRAX 10y Fx Prob (Orange-w/ DXA)",
8:"FRAX 10y Hip Fx Prob (Orange-w/ DXA)",
9:"Met Sx",
10:"AA_FLAG",
11:"L1_HU_BMD",
12:"TAT Area (cm2)",
13:"Total Body                Area EA (cm2)",
14:"VAT Area (cm2)",
15:"SAT Area (cm2)",
16:"VAT/SAT     Ratio",
17:"Muscle HU",
18:" Muscle Area (cm2)",
19:"L3 SMI (cm2/m2)",
20:"AoCa        Agatston",
21:"Liver HU    (Median)",
22:"CARD_FLAG",
23:"DIA_FLAG",
24:"FEM_FLAG",
25:"ALZ_FLAG",
26:"CANCER_FLAG",
27:"OUTCOME_FLAGS",
28:"DEATH_FLAG",


In [5]:
# define a lookup table for the column
columns = {0:"Clinical F/U interval  [d from CT]",
1:"BMI",
2:"BMI >30",
3:"Sex",
4:"Age at CT",
5:"Tobacco",
6:"FRS 10-year risk (%)",
7:"FRAX 10y Fx Prob (Orange-w/ DXA)",
8:"FRAX 10y Hip Fx Prob (Orange-w/ DXA)",
9:"Met Sx",
10:"AA_FLAG",
11:"L1_HU_BMD",
12:"TAT Area (cm2)",
13:"Total Body                Area EA (cm2)",
14:"VAT Area (cm2)",
15:"SAT Area (cm2)",
16:"VAT/SAT     Ratio",
17:"Muscle HU",
18:" Muscle Area (cm2)",
19:"L3 SMI (cm2/m2)",
20:"AoCa        Agatston",
21:"Liver HU    (Median)",
22:"CARD_FLAG",
23:"DIA_FLAG",
24:"FEM_FLAG",
25:"ALZ_FLAG",
26:"CANCER_FLAG",
27:"OUTCOME_FLAGS",
28:"DEATH_FLAG",}

# define different groups of column
clinical_data_columns = {i:columns[i] for i in range(11)}
CT_data_columns = {i:columns[i] for i in range(11,22)}
clinical_outcome_columns = {i:columns[i] for i in range(22,28)}

In [6]:

print(data.at[0,columns[2]]=='N')

False


In [7]:
for j in columns:
    col = columns[j]
    #for i in range(N):
    if not isinstance(data.at[0,col], (int, float)):
        print(f'{i} {col} {type(data.at[0,col])}')

DEATH_FLAG Clinical F/U interval  [d from CT] <class 'numpy.int64'>
DEATH_FLAG BMI >30 <class 'numpy.int64'>
DEATH_FLAG Sex <class 'numpy.int64'>
DEATH_FLAG Age at CT <class 'numpy.int64'>
DEATH_FLAG Tobacco <class 'numpy.int64'>
DEATH_FLAG AA_FLAG <class 'numpy.int64'>
DEATH_FLAG CARD_FLAG <class 'numpy.int64'>
DEATH_FLAG DIA_FLAG <class 'numpy.int64'>
DEATH_FLAG FEM_FLAG <class 'numpy.int64'>
DEATH_FLAG ALZ_FLAG <class 'numpy.int64'>
DEATH_FLAG CANCER_FLAG <class 'numpy.int64'>
DEATH_FLAG OUTCOME_FLAGS <class 'numpy.int64'>
DEATH_FLAG DEATH_FLAG <class 'numpy.int64'>


In [36]:
#N = len(data) - 1
N = 10

data_list = data.values.tolist()
for j in columns:
    col = columns[j]
    max_value = data[col].max()
    for i in range(N):
        data_list[i][j] += 1
        if max_value > 300:
            data_list[i][j] /= max_value

x_ct = [[data_list[i][j] for j in columns if j in CT_data_columns] for i in range(N)]
x_clinical = [[data_list[i][j] for j in columns if j in clinical_data_columns] for i in range(N)]
x_ctclinical = [[data_list[i][j] for j in columns if j in CT_data_columns or j in clinical_data_columns] for i in range(N)]
y_death = [data_list[i][28] for i in range(N)]
y_card = [data_list[i][22] for i in range(N)]
y_dia = [data_list[i][23] for i in range(N)]
y_fem = [data_list[i][24] for i in range(N)]
y_alz = [data_list[i][25] for i in range(N)]
y_cancer = [data_list[i][26] for i in range(N)]
y_outcome = [data_list[i][27] for i in range(N)]


'''
x_ct = [[data.at[i,columns[j]] for j in columns if j in CT_data_columns] for i in range(N)]
x_clinical = [[float(data.at[i,columns[j]]) for j in columns if j in clinical_data_columns] for i in range(N)]
x_ctclinical = [[data.at[i,columns[j]] for j in columns if j in CT_data_columns or j in clinical_data_columns] for i in range(N)]
y_death = [float(data.at[i,"DEATH_FLAG"]) for i in range(N)]
y_card = [data.at[i,"CARD_FLAG"] for i in range(N)]
y_dia = [data.at[i,"DIA_FLAG"] for i in range(N)]
y_fem = [data.at[i,"FEM_FLAG"] for i in range(N)]
y_alz = [data.at[i,"ALZ_FLAG"] for i in range(N)]
y_cancer = [data.at[i,"CANCER_FLAG"] for i in range(N)]
y_outcome = [data.at[i,"OUTCOME_FLAGS"] for i in range(N)]
'''

'\nx_ct = [[data.at[i,columns[j]] for j in columns if j in CT_data_columns] for i in range(N)]\nx_clinical = [[float(data.at[i,columns[j]]) for j in columns if j in clinical_data_columns] for i in range(N)]\nx_ctclinical = [[data.at[i,columns[j]] for j in columns if j in CT_data_columns or j in clinical_data_columns] for i in range(N)]\ny_death = [float(data.at[i,"DEATH_FLAG"]) for i in range(N)]\ny_card = [data.at[i,"CARD_FLAG"] for i in range(N)]\ny_dia = [data.at[i,"DIA_FLAG"] for i in range(N)]\ny_fem = [data.at[i,"FEM_FLAG"] for i in range(N)]\ny_alz = [data.at[i,"ALZ_FLAG"] for i in range(N)]\ny_cancer = [data.at[i,"CANCER_FLAG"] for i in range(N)]\ny_outcome = [data.at[i,"OUTCOME_FLAGS"] for i in range(N)]\n'

In [35]:
print(x_clinical)
        

[[0.0012289522826221511, 25.0, 0.0, 1.0, 60.0, 1.0, 0.06, 5.46, 0.75, 0.0, 0.0], [0.0002802411548885488, 22.4, 0.0, 1.0, 88.0, 1.0, 0.16, 11.65, 8.4, 0.0, 0.0], [0.0020312216038538425, 24.7, 0.0, 1.0, 68.0, 1.0, 0.1, 6.35, 1.59, 0.0, 0.0], [0.0010029683438116483, 20.0, 0.0, 0.0, 82.0, 1.0, 0.04, 24.03, 13.4, 0.0, 0.0], [0.0023441224422068464, 29.0, 0.0, 0.0, 76.0, 0.0, 0.04, 19.7, 7.26, 0.0, 0.0], [0.0022940793788338914, 27.9, 0.0, 1.0, 62.0, 1.0, 0.1, 5.01, 0.65, 0.0, 0.0], [0.002288811687952528, 37.8, 1.0, 1.0, 64.0, 1.0, 0.08, 3.87, 0.49, nan, 0.0], [0.000535197393546552, 29.8, 0.0, 1.0, 57.0, 0.0, 0.06, 3.96, 0.22, 0.0, 0.0], [0.0, 28.7, 0.0, 0.0, 62.0, 0.0, 0.01, 7.54, 0.69, 0.0, 0.0], [0.0013574839401274256, 25.8, 0.0, 0.0, 86.0, 0.0, 0.04, 21.46, 9.11, 0.0, 0.0]]


In [37]:
clf = LogisticRegression(random_state=0).fit(x_clinical, y_death)
clf.score(x_clinical, y_death)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').